In [135]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

data = pd.read_csv("animaldataset.csv")
data.head()

,Animal,Height (cm),Weight (kg),Color,Lifespan (years),Diet,Habitat,Predators,Average Speed (km/h),Countries Found,Conservation Status,Family,Gestation Period (days),Top Speed (km/h),Social Structure,Offspring per Birth
0,Aardvark,105-130,40-65,Grey,20-30,Insectivore,"Savannas, Grasslands","Lions, Hyenas",40,Africa,Least Concern,Orycteropodidae,210-240,40,Solitary,1
1,Aardwolf,40-50,8-14,Yellow-brown,10-12,Insectivore,"Grasslands, Savannas","Lions, Leopards",24-30,Eastern and Southern Africa,Least Concern,Hyaenidae,90,40,Solitary,2-5
2,African Elephant,270-310,2700-6000,Grey,60-70,Herbivore,"Savannah, Forest","Lions, Hyenas",25,Africa,Vulnerable,Elephantidae,640-660,40,Herd-based,1
3,African Lion,80-110,120-250,Tan,10-14,Carnivore,"Grasslands, Savannas","Hyenas, Crocodiles",58,Africa,Vulnerable,Felidae,98-105,80,Group-based,2-4 (usually)
4,African Wild Dog,75-80,18-36,Multicolored,10-12,Carnivore,Savannahs,"Lions, Hyenas",56,Sub-Saharan Africa,Endangered,Canidae,70,56,Group-based,10-12


In [136]:
# Checking for missing values
print(data.isnull().sum())

# Exploring the distribution of species
print(data["Animal"].value_counts())

# Visualizing the distribution of species
species_counts = data["Animal"].value_counts()
species_names = species_counts.index
species_quantity = species_counts.values

figure = px.pie(data, values=species_quantity, names=species_names, hole=0.5, title="Distribution of Species")
figure.show()


Animal                     0
Height (cm)                0
Weight (kg)                0
Color                      0
Lifespan (years)           0
Diet                       0
Habitat                    0
Predators                  0
Average Speed (km/h)       0
Countries Found            0
Conservation Status        0
Family                     0
Gestation Period (days)    0
Top Speed (km/h)           0
Social Structure           0
Offspring per Birth        0
dtype: int64
Animal
Pink Fairy Armadillo    2
Okapi                   2
Blobfish                2
Goblin Shark            2
Great White Shark       2
                       ..
Galápagos Tortoise      1
Galliformes             1
Gaur                    1
Gerenuk                 1
Zebra Shark             1
Name: count, Length: 196, dtype: int64


In [137]:
# Checking correlation (if applicable)
numeric_data = data.select_dtypes(include=['float64', 'int64'])
correlation = numeric_data.corr()
print(correlation)


Empty DataFrame
Columns: []
Index: []


In [138]:
# Creating a new target variable based on Habitat and Predator columns
data['Environmental_Threat'] = 'None'  # Initialize with 'None'
data.loc[(data['Habitat'].str.contains('Savanna') & data['Predators'].str.contains('Lion')), 'Environmental_Threat'] = 'Predation by Lions in Savanna'
data.loc[(data['Habitat'].str.contains('Forest') & data['Predators'].str.contains('Tiger')), 'Environmental_Threat'] = 'Predation by Tigers in Forest'
# Add more specific threats based on habitat and predator combinations

# Splitting the data for training
x = np.array(data.drop(columns=["Animal", "Habitat", "Predators", "Environmental_Threat"]))  
y = np.array(data[["Environmental_Threat"]])

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.20, random_state=42)


In [139]:
# Assuming xtrain is a NumPy array with multiple columns representing features
# If not, adjust the code to ensure that xtrain contains the required features
xtrain = pd.DataFrame(xtrain[:, :4], columns=["type", "amount", "oldbalanceOrg", "newbalanceOrig"])

# Convert categorical variables into dummy/indicator variables (one-hot encoding)
xtrain_encoded = pd.get_dummies(xtrain)

# Train the decision tree classifier using the encoded data
model = DecisionTreeClassifier()
model.fit(xtrain_encoded, ytrain)


DecisionTreeClassifier()

In [140]:
# Assuming xtest is a NumPy array with multiple columns representing features
# If not, adjust the code to ensure that xtest contains the required features
xtest = pd.DataFrame(xtest[:, :4], columns=["type", "amount", "oldbalanceOrg", "newbalanceOrig"])

# Convert categorical variables into dummy/indicator variables (one-hot encoding)
xtest_encoded = pd.get_dummies(xtest)

# Ensure that the columns in xtest_encoded match the columns in xtrain_encoded
missing_cols = set(xtrain_encoded.columns) - set(xtest_encoded.columns)
for col in missing_cols:
    xtest_encoded[col] = 0

# Reorder the columns to match the order in xtrain_encoded
xtest_encoded = xtest_encoded[xtrain_encoded.columns]

# Evaluate the model
print("Model Accuracy:", model.score(xtest_encoded, ytest))


C:\Users\danie\AppData\Local\Temp\ipykernel_26444\2886303441.py:11: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\danie\AppData\Local\Temp\ipykernel_26444\2886303441.py:11: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\danie\AppData\Local\Temp\ipykernel_26444\2886303441.py:11: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragment

Model Accuracy: 0.9024390243902439


In [141]:
# Assuming xtrain_encoded contains only the four features
# Retrain the decision tree classifier using only the four features
model = DecisionTreeClassifier()
model.fit(xtrain_encoded, ytrain)

# Ensure xtest has the same columns as xtrain_encoded
xtest_encoded = pd.get_dummies(xtest)
missing_cols = set(xtrain_encoded.columns) - set(xtest_encoded.columns)
for col in missing_cols:
    xtest_encoded[col] = 0

# Reorder the columns to match the order in xtrain_encoded
xtest_encoded = xtest_encoded[xtrain_encoded.columns]

# Make predictions using xtest_encoded
prediction = model.predict(xtest_encoded)
print("Predicted Environmental Threat:", prediction)


C:\Users\danie\AppData\Local\Temp\ipykernel_26444\1789464688.py:10: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\danie\AppData\Local\Temp\ipykernel_26444\1789464688.py:10: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\danie\AppData\Local\Temp\ipykernel_26444\1789464688.py:10: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragment

Predicted Environmental Threat: ['None' 'None' 'None' 'None' 'None' 'None' 'None' 'None'
 'Predation by Lions in Savanna' 'None' 'None' 'None' 'None' 'None'
 'Predation by Tigers in Forest' 'None' 'None' 'None' 'None' 'None' 'None'
 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None'
 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None']
